In [1]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install transformers datasets accelerate huggingface_hub
# !pip install vllm


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from vllm import LLM, SamplingParams
import os

# ---- Step 1: Load Tokenizer ----
model_name = "microsoft/phi-1_5"  # Or your model base
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ---- Step 2: Load Base Model (from HuggingFace) ----
base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)

# ---- Step 3: Load Checkpoint ----
ckpt = torch.load("fsdp_model_checkpoint.pt", map_location="cpu")

# ---- Step 4: Replace LM head with one trained head ----
# Assuming the head you want to use is the first one (index 0)
selected_head_weight = ckpt["heads.0.weight"]
selected_head_bias = ckpt["heads.0.bias"]

# Load the weights into the base model's LM head
with torch.no_grad():
    base_model.lm_head.weight.copy_(selected_head_weight)
    if base_model.lm_head.bias is not None:
        base_model.lm_head.bias.copy_(selected_head_bias)

# ---- Step 5: Save HuggingFace-style model for vLLM ----
save_path = "./dpo_singlehead_phi1_5"
base_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3/dist-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 677, in start
    s

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [4]:
from datasets import load_from_disk

# Step 1: Load the test dataset
test_dataset = load_from_disk("cached_split/test")

# Step 2: Construct full prompts
prompts = [
    f"{example['system']}\n{example['prompt']}"
    for example in test_dataset
]

# Optional: limit to first N prompts
prompts = prompts[:10]
for i, p in enumerate(prompts):
    print(f"[Prompt {i}]\n{p}\n{'='*40}")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
from vllm import LLM, SamplingParams

# Load from saved single-head model
llm = LLM(model=save_path, dtype="auto")  # make sure GPU is available

# Define prompts
# prompts = [
#     "System: You are a helpful assistant.\nUser: What's the capital of France?",
#     "System: You are a wise tutor.\nUser: Explain the concept of entropy in simple terms."
# ]

sampling_params = SamplingParams(temperature=0.7, max_tokens=100)

# Run inference
outputs = llm.generate(prompts[:10], sampling_params)

# Display results
for i, output in enumerate(outputs):
    print(f"Prompt {i}:")
    print(f"Generated: {output.outputs[0].text.strip()}\n{'='*40}")

In [ ]:
# !python -c "import torch; print(torch.__version__)"

In [ ]:
# !pip uninstall -y torch vllm

# !pip install torch==2.2.2+cu118 torchvision --index-url https://download.pytorch.org/whl/cu118

# !pip install vllm
